<a href="https://colab.research.google.com/github/Ash100/DaS/blob/main/DFT_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I am **Dr. Ashfaq Ahmad**, working in the field of Bioinformatics and Structure Biology. This notebook is designed from the published work of Jacob S. Hirschi and can be found and cited from here *https://doi.org/10.1021/acs.jchemed.3c00535*

If you are interested in learning with me, you can subscribe and join my youtube channel here **https://www.youtube.com/@Bioinformaticsinsights**

# Introduction

Now that you have some intuition for the analytic 3D PIB solutions from **Notebook 1** and the spatial orientation of the $\pi$ electrons in PAHs from **Notebook 2**, it is time to account for electron-electron interaction in the PIB model through Density-Functional Theory (DFT) using a real-space grid basis. We'll consider each component of the total DFT energy individually in the Kohn-Sham scheme then combine everything into a DFT calculator at the end.

In [ ]:
#@title Install/Import Packages

# import standard anaconda packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%config InlineBackend.figure_format = 'svg'
from scipy import sparse
from IPython.display import Markdown, display, clear_output
import ipywidgets as widgets
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Keep this commented out until ipyvolume is fixed
#try:
#  import ipyvolume as ipv
#except:
#  !pip install -q ipyvolume==v0.6.3
#  import ipyvolume as ipv

from google.colab import output
output.enable_custom_widget_manager()

# Background Theory
The justification for performing DFT calculations comes from two theorems by Pierre Hohenberg and Walter Kohn published in [1964](https://journals.aps.org/pr/abstract/10.1103/PhysRev.136.B864). They state, effectively, that <br>
(1) there is a one-to-one mapping between the electron density and external potential and   
(2) a variational principle exists for the electron density. <br>
According to Hohenberg and Kohn, instead of directly solving the Schr&ouml;dinger equation to find the ground state energy, there is a strong theoretical basis for finding it through the density:

$$E[n(r)] = \int n(r) v_{\text{ext}}(r) dr + F[n(r)] $$

where $F[n(r)]$ is the unknowable universal functional, $n(r)$ is the electron density, and $v_{\text{ext}}(r)$ is the external potential. The Kohn-Sham (KS) method of DFT, proposed in [1965](https://journals.aps.org/pr/abstract/10.1103/PhysRev.140.A1133), practicalizes these equations by introducing an invented system of non-interacting particles obeying the single-particle Hamiltonian equation

$$\hat{h}_{\text{KS}} \phi_i(r) = \Big[-\frac{1}{2} \nabla_i^2 + v_{\text{ext}}(r) + v_{\text{Ha}}(r) + v_{\text{XC}}(r) \Big] \phi_i(r) = \epsilon_i \phi_i(r)$$

where $v_{\text{Ha}}(r)$ is the Hartree potential and $v_{\text{XC}}(r)$ is the exchange-correlation potential. The eigenstate solutions of this equation can be used to find a density that, by its construction, reproduces the exact density of the full interacting system:

$$n(r)=2\sum_i |\phi_i^{\text{KS}}(r)|^2.$$
The total energy in the KS approach is a functional of the electron density and has the form:

$$ E_{\text{tot}}[n(r)] = T_{s}[n(r)] + E_{\text{ext}}[n(r)] + E_{\text{Ha}}[n(r)] + E_{\text{XC}}[n(r)].$$

Now we'll go through the mechanics of each term in KS potential, $\hat{h}_{\text{KS}}$, and total energy functional, $E_{\text{tot}}[n(r)]$.

## Grid and Kinetic Energy

To start, we'll consider the kinetic energy operator in cartesian coordinates   with atomic units:

$$\hat{T}=-\frac{1}{2}\nabla^2=-\frac{1}{2}\frac{d^2}{dx^2}-\frac{1}{2}\frac{d^2}{dy^2}-\frac{1}{2}\frac{d^2}{dz^2}.$$

The finite difference method provides a numerical way for approximating the 1st, 2nd, 3rd, ... derivatives of functions by evaluating the neighbouring points. For a function $f$ in the $x$-dimension this would have the form

$$\frac{d^2f _{i,j,k}}{dx^2}=\frac{f _{i+1,j,k}-2f _{i,j,k}+f _{i-1,j,k}}{\Delta x^2}$$

or in a matrix representation:

$$\frac{d^2}{dx^2}=\frac{1}{\Delta x^2}\begin{bmatrix}
-2 & 1 & 0 & \dots & 0\\
1 & -2 & 1 & \dots & 0 \\
0 & 1 & -2 & \dots & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & \dots & -2 \\
\end{bmatrix}$$

$$\rightarrow \hat{T}_x = - \frac{1}{2} \frac{d^2}{dx^2}. $$

Each $x, y, z$ component of the kinetic operator is a square matrix in which the number of elements depends on the grid points in that direction. In three dimensions the overall operator can be expressed with a kronecker sum:

$$ \hat{T}_{\text{tot}} =\hat{T_x}\bigoplus\hat{T_y}\bigoplus\hat{T_z} =
\begin{bmatrix}
[\hat{T}_x]  &       0      &      0      \\
0            & [\hat{T}_y]  &      0      \\
0            &       0      & [\hat{T}_z]  \\
\end{bmatrix}
$$

$\hat{T}_{\text{tot}}$ is a diagonally dominant sparse matrix with (grid points)$^2$ entries.

In [ ]:
#@title **Kinetic Energy**

# defining the GUI widgets
lx_slider          = widgets.IntSlider(value=12,min=3,max=32,step=1,description='lx',disabled=False,readout_format='d',continuous_update=False)
ly_slider          = widgets.IntSlider(value=8,min=3,max=32,step=1,description='ly',disabled=False,readout_format='d',continuous_update=False)
lz_slider          = widgets.IntSlider(value=3,min=3,max=32,step=1,description='lz',disabled=False,readout_format='d',continuous_update=False)
length_labels      = widgets.Label(value='Box Lengths (Bohr): ')
box_length_ui      = widgets.HBox([length_labels, lx_slider, ly_slider, lz_slider],layout=widgets.Layout(border='solid 2px',width='%50'))
num_elect_dropdown = widgets.Dropdown(options=np.arange(2,31,2),value=10,description='electrons:',disabled=False)
k_en_button        = widgets.Button(description='Calculate Kinetic Eigenstates',layout=widgets.Layout(width='auto'))
state_selector     = widgets.IntSlider(value=2,min=0,max=50,step=1,description='state',disabled=False,readout_format='d',continuous_update=False)
psi_square_box     = widgets.Checkbox(value=False, description='psi square', disabled=False)
density_button     = widgets.Button(description='Calculate Density',layout=widgets.Layout(width='auto'))
lib_dropdown_grid  = widgets.Dropdown(options=[('ipyvolume', 'ipyvol'), ('plotly', 'plotly')], value='plotly',
                                description='3D Plotting Library: ', disabled=False, layout = widgets.Layout(width='225px'),
                                style = {'description_width': 'initial'})
lib_dropdown_grid.disabled       = True
k_en_button.style.font_weight    = 'bold'
density_button.style.font_weight = 'bold'

def edge_cleaner(func_3d, nx, ny, nz, num_edges=1):
  '''Sets outermost layer (or two) of grid values of a 3D function to zero.
  This is needed for the density because of the discontinuities in the numeric 2nd derivatives at the box edges.
  Args:
    func_3d (np.array): Flattened 3D grid of points.
    nx, ny, nz (int): Number of grid points in each dimension.
    num_edges (1 or 2): Number of border layers to set to zero.
  Returns:
    A flattened array of grid points.
  '''
  func_3d = func_3d.reshape(nx, ny, nz)
  if num_edges == 1:
    func_3d[0,:,:]  = 0
    func_3d[-1,:,:] = 0
    func_3d[:,0,:]  = 0
    func_3d[:,-1,:] = 0
    func_3d[:,:,0]  = 0
    func_3d[:,:,-1] = 0
  elif num_edges == 2:
    func_3d[0,:,:]  = 0
    func_3d[-1,:,:] = 0
    func_3d[:,0,:]  = 0
    func_3d[:,-1,:] = 0
    func_3d[:,:,0]  = 0
    func_3d[:,:,-1] = 0
    func_3d[1,:,:]  = 0
    func_3d[-2,:,:] = 0
    func_3d[:,1,:]  = 0
    func_3d[:,-2,:] = 0
    func_3d[:,:,1]  = 0
    func_3d[:,:,-2] = 0
  return func_3d.flatten()

def integ_3d(func_3d, dx, dy, dz):
  '''Integrates a 3D function over all defined space.
  Args:
    func_3d (np.array): 3D grid of points.
    dx, dy, dz (float): Differential volume element in each dimension.
  Returns:
    Integrated 3D function as scalar value.
  '''
  return np.sum(func_3d * dx * dy * dz)

def norm_psi_and_den(e_vecs, occ_states, dx, dy, dz):
  '''Normalizes raw eigenvectors from the solver and finds electron density.
  Args:
    e_vecs (np.array): Array of eigenvectors from the eigenvalue solver.
    occ_states (int): Number of occupied KS states (i.e. # electrons / 2).
    dx, dy, dz (float): Differential volume element in each dimension.
  Returns:
    norm_psi (np.array): Array of normalized eigenvectors.
    el_den (np.array): Electron density as array.
  '''
  norm_psi = np.zeros_like(e_vecs)
  el_den   = np.zeros_like(e_vecs[:,0])
  for i in range(e_vecs.shape[1]):
      norm_psi[:,i] = e_vecs[:,i]/np.sqrt(integ_3d(e_vecs[:,i]**2, dx, dy, dz))
  for i in range(occ_states):
      el_den += 2* norm_psi[:,i]**2
  return norm_psi, el_den

def noninter_kin_e(norm_eigenvecs, occ_states, kin_mat, dx, dy, dz, nx, ny, nz):
  '''Finds noninteracting KS kinetic energy.
  Args:
    norm_eigenvecs (np.array): Array of normalized eigenvectors as columns/rows.
    occ_states (int): Number of occupied KS states (i.e. # electrons / 2).
    kin_mat (sparse array): Kinetic energy operator as matrix.
    dx, dy, dz (float): Differential volume element in each dimension.
    nx, ny, nz (int): Number of grid points in each dimension.
  Returns:
    Scalar energy value in Ha.
  '''
  kin_energy_values = []
  for eig in norm_eigenvecs.T[:occ_states]:
    inner_prod   = eig*kin_mat.dot(eig)
    inner_prod   = edge_cleaner(inner_prod, nx, ny, nz, num_edges=1)
    orbital_k_en = integ_3d(inner_prod, dx, dy, dz)
    kin_energy_values.append(orbital_k_en)
  return sum(kin_energy_values)

def grid_density(l_x, l_y, l_z, plotting_lib='plotly'):
  '''Generates 3D scatter plot representing grid for DFT calculations.
  Args:
    l_x, l_y, l_z (int): Box lengths in each direction.
  '''
  nx, ny, nz = (5 * l_x), (5 * l_y), (5 * l_z)
  gpoints    = (nx * ny * nz)
  xp, yp, zp = np.linspace(0, l_x, nx), np.linspace(0, l_y, ny), np.linspace(0, l_z, nz)
  X, Y, Z    = np.meshgrid(xp, yp, zp, indexing='ij')
  label      = '### <center> Grid Points (5 pts/bohr): '
  label     += '$x_p \\times y_p \\times z_p ='
  label     += f'{nx}' + '\\times' + f'{ny}' + '\\times' + f'{nz}' + ' = '+ f'{gpoints}$ <center/>'
  display(Markdown(label))
  if plotting_lib == 'ipyvol':
    fig1 = ipv.figure(title='PIB',width=450, height=450)
    fig1.camera.type = 'OrthographicCamera'
    ipv.quickscatter(X.flatten(), Y.flatten(), Z.flatten(), size=0.5, marker='box',description='grid points')
    ipv.squarelim()
    ipv.style.box_off()
    ipv.show()
  elif plotting_lib == 'plotly':
    scatter = go.Figure(data=[go.Scatter3d(x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
                                       mode='markers',
                                       marker=dict(size=2,color='red',symbol='square',opacity=0.5))])
    scatter.update_layout(scene = dict(xaxis_title='x', yaxis_title='y', zaxis_title='z',
                  aspectmode='data'), width=900, height=500)
    scatter.show()

def kin_en_eigenstates(b):
  '''Finds non-interacting kinetic energy eigenstates using parameters from the widgets.
  Args:
    b : button click
  '''
  occ_states = int(num_elect_dropdown.value/2)
  nextra = 2 # two unoccupied states calculated in addition to occupied (arbitrary number)

  # construct 3D grid of points
  nx, ny, nz = (5 * lx_slider.value), (5 * ly_slider.value), (5 * lz_slider.value)
  xp, yp, zp = np.linspace(0, lx_slider.value, nx), np.linspace(0, ly_slider.value, ny), np.linspace(0, lz_slider.value, nz)
  X, Y, Z    = np.meshgrid(xp, yp, zp, indexing='ij')

  diag1x = np.ones(nx)/(xp[1])
  diag1y = np.ones(nx)/(yp[1])
  diag1z = np.ones(nx)/(zp[1])

  # calculation of d/dx, d/dy, and d/dz as sparse matrices:
  D1x    = sparse.spdiags(np.array([-diag1x, diag1x]), np.array([0,1]), nx, nx)
  D1y    = sparse.spdiags(np.array([-diag1y, diag1y]), np.array([0,1]), ny, ny)
  D1z    = sparse.spdiags(np.array([-diag1z, diag1z]), np.array([0,1]), nz, nz)

  # overall 1st derivative for functions on this grid:
  D1st   = sparse.kronsum(D1z,sparse.kronsum(D1y,D1x))

  diagx  = np.ones(nx)/(xp[1]**2)
  diagy  = np.ones(ny)/(yp[1]**2)
  diagz  = np.ones(nz)/(zp[1]**2)

  # calculation of d^2/dx^2, d^2/dy^2, and d^2/dz^2 as sparse matrices:
  Dx     = sparse.spdiags(np.array([diagx, -2*diagx, diagx]), np.array([-1,0,1]), nx, nx)
  Dy     = sparse.spdiags(np.array([diagy, -2*diagy, diagy]), np.array([-1,0,1]), ny, ny)
  Dz     = sparse.spdiags(np.array([diagz, -2*diagz, diagz]), np.array([-1,0,1]), nz, nz)

  # sparse matrix representing kinetic energy:
  T      = -1/2 * sparse.kronsum(Dz, sparse.kronsum(Dy, Dx))

  # finds the eigenvalues and eigenvectors (k is number of levels)
  free_p_eval, free_p_evec = sparse.linalg.eigsh(T, k=occ_states+nextra, which='SM',mode='cayley')

  normalized_psi, density = norm_psi_and_den(free_p_evec, occ_states, xp[1], yp[1], zp[1])

  # grid is now set; rerun whole cell to reset
  lx_slider.disabled          = True
  ly_slider.disabled          = True
  lz_slider.disabled          = True
  num_elect_dropdown.disabled = True
  k_en_button.disabled        = True

  def state_plotter(state_num, psi_square, plotting_lib='plotly'):
    '''Generates interactive 3D rendering of kinetic energy eigenstates.
    Args:
      state_num (int): State number to consider, i.e. 0, 1, 2, ...
      psi_square (bool): Show orbital (False) or probability density (True).
    '''
    state_selector.max = occ_states + nextra - 1
    occ_list = 2*np.ones(occ_states)
    occ_list = np.append(occ_list, np.zeros(nextra))
    state_info = f'Orbital Energy, $\epsilon_{state_num}$ = {free_p_eval[state_num]:.3f} Ha'
    state_info += f"   |   'Occupancy': {occ_list[state_num]}"
    display(Markdown(state_info))
    #isoslider.min,  = np.round(normalized_psi.T[state_num].min(), 3)
    #isoslider.max   = np.round(normalized_psi.T[state_num].max(), 3)
    if plotting_lib == 'ipyvol':
      ipv.clear()
      fig2 = ipv.figure(title='Eigenstates',width=450, height=450)
      fig2.camera.type = 'OrthographicCamera'
      if state_num == 0 and not psi_square:
        ipv.pylab.plot_isosurface(normalized_psi.T[state_num].reshape(nx, ny, nz),
                                  color='red', # uses default isovalue
                                  controls=True,
                                  description='positive')
      elif state_num == 0 and psi_square:
        ipv.pylab.plot_isosurface(normalized_psi.T[state_num].reshape(nx, ny, nz)**2,
                            color='red', # uses default isovalue
                            controls=True,
                            description='prob. density')
      ## isodensity surface; psi_square = True
      elif psi_square:
        iso_level = normalized_psi.T[state_num].max()**(7/2)
        ipv.pylab.plot_isosurface(normalized_psi.T[state_num].reshape(nx, ny, nz)**2,
                                  color='red', level=iso_level,
                                  controls=True,
                                  description='prob. density')

      ## positive and negative isosurfaces;
      else:
        value_array = normalized_psi.T[state_num].reshape(nx, ny, nz)
        pos_sur = ipv.pylab.plot_isosurface(value_array,
                                            color='red',
                                            level=value_array.max()**(2)*1.5, controls=True,
                                            description='positive')
        neg_sur = ipv.pylab.plot_isosurface(value_array,
                                            color='blue',level=-value_array.max()**(2)*1.5,
                                            controls=True,
                                            description='negative')
      ipv.squarelim()
      ipv.style.box_off()
      ipv.show()

    elif plotting_lib == 'plotly':
      if state_num == 0 and not psi_square:
        prob_wf_iso = normalized_psi.T[state_num].mean()
        den_fig = go.Figure(data=go.Isosurface(
        x=X.flatten(),
        y=Y.flatten(),
        z=Z.flatten(),
        value=normalized_psi.T[state_num].flatten(),
        colorscale='BlueRed',
        isomin=-prob_wf_iso,
        isomax=prob_wf_iso,
        surface_count=2,
        showscale=False,
        caps=dict(x_show=False, y_show=False, z_show=False)))
        den_fig.update_layout(scene = dict(
                          xaxis_title='x',
                          yaxis_title='y',
                          zaxis_title='z',
                          aspectmode='data'),
                          width=800,
                          height=450)
        den_fig.show()

      elif state_num == 0 and psi_square:
        prob_den_iso = normalized_psi.T[state_num].max()**(7/2)
        den_fig = go.Figure(data=go.Isosurface(
        x=X.flatten(),
        y=Y.flatten(),
        z=Z.flatten(),
        value=normalized_psi.T[state_num].flatten()**2,
        colorscale='BlueRed',
        isomin=-prob_den_iso,
        isomax=prob_den_iso,
        surface_count=2,
        showscale=False,
        caps=dict(x_show=False, y_show=False, z_show=False)))
        den_fig.update_layout(scene = dict(
                          xaxis_title='x',
                          yaxis_title='y',
                          zaxis_title='z',
                          aspectmode='data'),
                          width=800,
                          height=450)
        den_fig.show()

      ## isodensity surface; psi_square = True
      elif psi_square:
        prob_den_iso = normalized_psi.T[state_num].max()**(7/2)
        den_fig = go.Figure(data=go.Isosurface(
        x=X.flatten(),
        y=Y.flatten(),
        z=Z.flatten(),
        value=normalized_psi.T[state_num].flatten()**2,
        colorscale='BlueRed',
        isomin=-prob_den_iso,
        isomax=prob_den_iso,
        surface_count=2,
        showscale=False,
        caps=dict(x_show=False, y_show=False, z_show=False)))
        den_fig.update_layout(scene = dict(
                          xaxis_title='x',
                          yaxis_title='y',
                          zaxis_title='z',
                          aspectmode='data'),
                          width=800,
                          height=450)
        den_fig.show()

      ## positive and negative isosurfaces;
      else:
        prob_wf_iso = normalized_psi.T[state_num].max()**(2)*1.5
        den_fig = go.Figure(data=go.Isosurface(
        x=X.flatten(),
        y=Y.flatten(),
        z=Z.flatten(),
        value=normalized_psi.T[state_num].flatten(),
        colorscale='BlueRed',
        isomin=-prob_wf_iso,
        isomax=prob_wf_iso,
        surface_count=2,
        showscale=False,
        caps=dict(x_show=False, y_show=False, z_show=False)))
        den_fig.update_layout(scene = dict(
                          xaxis_title='x',
                          yaxis_title='y',
                          zaxis_title='z',
                          aspectmode='data'),
                          width=800,
                          height=450)
        den_fig.show()

  def plot_inter_density(b):
    '''Generates 3D rendering of noninteracting electron density.
    Also calculates noninteracting KS kinetic energy in Ha.
    Args:
      b : Button click.
    '''
    density_button.disabled = True
    kin_ener_value     = noninter_kin_e(normalized_psi, occ_states, T, xp[1], yp[1], zp[1], nx, ny, nz)
    kin_ener_equation  = 'KS Kinetic Energy: '
    kin_ener_equation += '$T_s[n(r)] = - \\frac{1}{2} \sum_i^{N}\langle \phi_i^{\\text{KS}}(r) |  \\nabla^2|\phi_i^{\\text{KS}}(r)\\rangle = '
    kin_ener_equation += f'{kin_ener_value:.3f}$ Ha'
    display(Markdown(kin_ener_equation))
    display(Markdown('''If there were no e<sup>-</sup>-e<sup>-</sup> interaction, this
     would be the complete ground-state solution and this energy would be the total
      energy of the system, directly comparable to
    the results from Notebook 1 in the limit of infinite grid density.
    In the following cells, we'll consider the other terms in the Hamiltonian that
    depend on e<sup>-</sup>-e<sup>-</sup> interaction.'''))
    if lib_dropdown_grid.value == 'ipyvol':
      ipv.clear()
      fig3 = ipv.figure(title='Density',width=450, height=450)
      fig3.camera.type = 'OrthographicCamera'
      ipv.style.box_off()
      ipv.plot_isosurface(density.reshape(nx, ny, nz),
                          color='red',
                          level=density.mean()*2,
                          description='density')
      ipv.squarelim()
      ipv.style.box_off()
      ipv.show()
    elif lib_dropdown_grid.value == 'plotly':
      den_fig = go.Figure(data=go.Isosurface(
      x=X.flatten(),
      y=Y.flatten(),
      z=Z.flatten(),
      value=density.flatten(),
      colorscale='BlueRed',
      isomin=-density.mean()*2,
      isomax=density.mean()*2,
      surface_count=2,
      showscale=False,
      caps=dict(x_show=False, y_show=False, z_show=False)))
      den_fig.update_layout(scene = dict(
                        xaxis_title='x',
                        yaxis_title='y',
                        zaxis_title='z',
                        aspectmode='data'),
                        width=800,
                        height=450)
      den_fig.show()

    display(Markdown(f'isovalue: {density.mean()*2:.4f} (e/Bohr$^3$)'))

  density_button.on_click(plot_inter_density) # link button + function

  state_output = widgets.interactive_output(state_plotter, {'state_num': state_selector,
                                                            'psi_square': psi_square_box})

  current_state = state_selector.value

  display(state_selector,
          psi_square_box,
          state_output,
          Markdown('---'),
          Markdown('## Non-interacting Density'),
          Markdown('The occupied Kohn-Sham eigenstates can be used to determine the non-interacting density, $n(r) = 2  \sum_i^{N}|\phi_i^{\\text{KS}}(r)|^2$, and kinetic energy, $T_s[n(r)] = - \\frac{1}{2} \sum_i^{N}\langle \phi_i^{\\text{KS}}(r) |  \\nabla^2|\phi_i^{\\text{KS}}(r)\\rangle$.'),
          widgets.HBox([num_elect_dropdown, density_button]))

k_en_button.on_click(kin_en_eigenstates) # link button to function

# link sliders to function
grid_out = widgets.interactive_output(grid_density, {'l_x': lx_slider,
                                                     'l_y': ly_slider,
                                                     'l_z': lz_slider})

# display the GUI layout
display(lib_dropdown_grid,
        Markdown('## Build Discrete Grid'),
        Markdown('''Most DFT codes perform integrals on numerical grids. Below
        we'll construct a 3D mesh by defining three box lengths and
        allocating 5 grid points per Bohr. The resultant
        grid can be visualized by representing each point as a red cube.'''),
        box_length_ui,
        grid_out,
        Markdown('---'),
        Markdown('## Kinetic Energy Eigenstates'),
        Markdown('''With the above defined grid,
        we can write a matrix that represents the kinetic energy operator
        acting on numeric functions on this grid. The eigenvalues
        and eigenvectors of this matrix represent the non-interacting solutions.'''),
        Markdown('$\hat{T}_{\\text{tot}} = \hat{T}_z\\bigoplus\hat{T}_y\\bigoplus\hat{T}_z \\xrightarrow[\\text{linalg.eigsh}]{\\text{scipy}} |\phi_i(r)\\rangle,\ \epsilon_i$'),
        widgets.HBox([num_elect_dropdown,k_en_button, widgets.Label(value='(This takes a few seconds.)')])
)

# LDA Exchange

The simplest approach to electronic exchange
is the Local Density Approximation (LDA), whereby the exchange
energy per particle at each point in space is assumed to be equal to that of
a uniform electron gas (UEG) with the same density. The LDA exchange potential has a
concise form found by Dirac in [1930](https://www.cambridge.org/core/journals/mathematical-proceedings-of-the-cambridge-philosophical-society/article/note-on-exchange-phenomena-in-the-thomas-atom/6C5FF7297CD96F49A8B8E9E3EA50E412): \\
\
$$ v_{\text{X}}^{\text{LDA}}(n(r)) = -\frac{3}{4}\left(\frac{3}{\pi}\right)^{1/3} n(r)^{1/3} .$$
\
This potential can be integrated against the density to yield the total
exchange energy,
$ E_{\text{X}}^{\text{LDA}}[n(r)] = \int{v_{\text{X}}^{\text{LDA}}(n(r))n(r)} dr .$
The LDA is a local functional in the sense that it only depends on the
value of the density at each point (grid point in our case), as opposed to
the integrals over all space required for Hartree-Fock exchange.


In [ ]:
#@title **Exchange Diagram**

# density GUI widget
LDA_exch_slider = widgets.FloatSlider(description='n(r)', min=0.00, max=2, step=0.05, continuous_update=False)

def exch_equation(den_number):
  '''Displays LaTeX equation of LDA exchange potential.
  Args:
    den_number (float): Value of electron density, n(r).
  '''
  exch_pot_number = np.round(-(3/4)*(3/np.pi)**(1/3)*den_number**(1/3), decimals=3)
  text = Markdown('## $v_{\\text{X},\ n(r)= \ ' + f'{den_number:.2f}' + '}^{\\text{LDA}} = -\\frac{3}{4}' +
                       '\left(\\frac{3}{\pi}\\right)^{1/3}' +
                       f'({den_number:.2f})' + '^{1/3}' +
                       f'={exch_pot_number:.3f}\ ' + '\\text{Ha/e}$')
  display(text)
  clear_output(wait=True)

def exch_pot_eq(den_number):
  '''Displays line plot of LDA exchange with scatterpoint at density value.
  Args:
    den_number (float): Value of electron density, n(r).
  '''
  exch_pot_number = np.round(-(3/4)*(3/np.pi)**(1/3)*den_number**(1/3), decimals=3)
  display(Markdown('<br>'))
  x = np.linspace(0, 10, 300)
  y = np.round(-(3/4)*(3/np.pi)**(1/3)*x**(1/3), decimals=3)
  fig = plt.figure()
  ax  = fig.add_subplot(1, 1, 1)
  plt.cla()
  plt.clf()
  clear_output(wait=True)
  plt.plot(x, y, label=r'Slater exchange')
  plt.hlines(exch_pot_number, 0, den_number, colors='black')
  plt.vlines(den_number, -2, exch_pot_number, colors='black')
  plt.scatter(den_number, exch_pot_number, marker="s", color='red', label='grid point')
  plt.title('LDA Exchange Potential', size=15)
  plt.xlabel('Density, $n(r)$', size=15)
  plt.ylabel('Potential, $v_{\mathrm{X}}^{\mathrm{LDA}}(n(r))$', size=15)
  plt.xlim(0,2)
  plt.ylim(-1.25,0)
  plt.rcParams["legend.markerscale"] = 1.5
  plt.legend(loc='upper right', fontsize=15)
  plt.show()

# link slider to functions
LDA_exc_output  = widgets.interactive_output(exch_pot_eq, {'den_number':LDA_exch_slider})
equation_output = widgets.interactive_output(exch_equation, {'den_number':LDA_exch_slider})

# generate the display
display(LDA_exch_slider,
        equation_output,
        LDA_exc_output)

# LDA Correlation
The correlation energy from a UEG
does not have an exact form applicable to densities in the intermediate regime
and the fitted parameterizations tend to be very complicated. However, an
expression found by [Chachiyo](https://pubs.aip.org/aip/jcp/article/145/2/021101/907773/Communication-Simple-and-accurate-uniform-electron) in 2016 with only two fitting parameters that accurately reproduces the UEG results does have a simple, elegant form:
\
$$v_{\text{C}}^{\text{LDA}}(r_s)= a\cdot \ln\left(1+\frac{b}{r_s}+\frac{b}{r_s^2}\right)$$
where $a = \frac{\ln2-1}{2\pi^2}$, $b \approx 20.4562557$, and $r_s$ is the
density-dependent Wigner-Seitz radius,
$r_s = \left(\frac{3}{4\pi n(r)}\right)^{1/3}$. $r_s$ is proportional to the inverse cube root of the density and has the following limits: <br>
<center>
low-density, $n(r) \rightarrow 0 \Longrightarrow r_s \rightarrow \infty $ <br>
high-density,  $n(r) \rightarrow \infty \Longrightarrow r_s \rightarrow 0 $.
<center/>

In [ ]:
#@title **Correlation Diagram**

# r_s GUI widget
LDA_cor_slider = widgets.FloatSlider(description='r_s', min=0.01, max=10, step=0.01,
                                      continuous_update=False)
def LDA_c_display(ws_radius):
  '''Displays line plot of Chachiyo LDA correlation potential with a
  scatterpoint and LaTeX equation at the Wigner-Seitz radius (r_s) value.
  Args:
    ws_radius (float): Wigner-Seitz radius (inversely related to density).
  '''
  # a, b, c fitting constants used in paper: 10.1063/1.4958669
  a, b, c = (np.log(2)-1)/(2*np.pi**2), 20.4562557, (4*np.pi/3)**(1/3)
  lda_expression = lambda rad: a*np.log(1 + b*c*1/(rad) + b*(c**2)*1/(rad))
  cor_pot_number = lda_expression(ws_radius)

  # LaTeX Equation
  text = Markdown('## $v_{\\text{C},\ r_s=' + f'{ws_radius:.2f}' + '}^{\\text{LDA}}' + '= a\cdot \\text{ln}\left(1+\\frac{b}{' +
                f'({ws_radius:.2f})' + '} + \\frac{b}{' + f'({ws_radius:.2f})^2' +
                '}\\right)=' + f'{cor_pot_number:.3f}\ ' + '\\text{Ha/e}$')
  display(text)

  # Matplotlib Diagram
  x = np.linspace(0.001, 10, 100)
  y = lda_expression(x)
  plt.plot(x, y, label='Chachiyo Correlation')
  plt.scatter(ws_radius, cor_pot_number, color='red', marker="s", label='grid point')
  plt.hlines(cor_pot_number, 0, ws_radius, colors='black')
  plt.vlines(ws_radius, -2, cor_pot_number, colors='black')
  plt.title('LDA Correlation Potential', size=15)
  plt.xlabel('Wigner-Seitz Radius, $r_s$', size=15)
  plt.ylabel('Potential, $v_{\mathrm{C}}^{\mathrm{LDA}}(r_s)$', size=15)
  plt.xlim(0,10)
  plt.ylim(-0.15,0)
  plt.rcParams["legend.markerscale"] = 1.5
  plt.legend(loc='upper right', fontsize=15)
  plt.show()

# link slider to function
output = widgets.interactive_output(LDA_c_display, {'ws_radius':LDA_cor_slider})

# generate the display
display(LDA_cor_slider, output)

# PBE Exchange

Another approximate approach to exchange-correlation functionals is the Generalized Gradient Approximation (GGA), which incorporates information not only about the density at each point in space but also
the gradient, or 1st derivative. The most popular GGA nowadays was reported in a [1996 paper](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.77.3865) by Perdue, Burke, and Ernzerhof and is acronymized as PBE. The electronic
exchange potential in PBE is defined relative to LDA exchange: \\
\
$$ v_{\text{X}}^{\text{PBE}}(n(r),s) = v_{\text{X}}^{\text{LDA}}(n(r))F_x(s) $$
\
where the $F_x(s)$ term is an exchange enhancement factor that has the form \
\
$$F_x(s)= 1+ \kappa -\frac{\kappa}{\left(1+\frac{\mu s^2}{\kappa}\right)}.$$ \

$F_x(s)$ depends on two constants, $\kappa = 0.804$ and $\mu \approx 0.21951 $, and the unitless reduced density gradient (RDG), \\
\
$$s = \frac{|\nabla{n(r)}|}{2(3)^{1/3}\pi^{2/3}n(r)^{4/3}} .$$


In [ ]:
#@title **Exchange Diagram**

# GUI density widgets
GGAx_den_slider  = widgets.FloatSlider(description='n(r)', min=0.05, max=1, step=0.05,
                                      continuous_update=False)
GGAx_grad_slider = widgets.FloatSlider(description='|grad n(r)|', min=0.01, max=2, step=0.05,
                                      continuous_update=False)

def GGAx_pot_eq(den_num, grad_den_num):
  '''Generates line plot of PBE exchange enhancement factor with a LaTeX equation
   and scatterpoint at the specified density + gradient value.
  Args:
    den_num (float): Value of the electron density.
    grad_den_num (float): Gradient of electron density at same point.
  '''
  kappa, mu = 0.804, 0.21951 # constants used in the paper, 10.1103/PhysRevLett.77.3865
  # reduced density gradient, s:
  calculate_s   = lambda den, grad_den: abs(grad_den)/(2*3**(1/3)*np.pi**(2/3)*den**(4/3))
  # enhancement factor, F_s:
  calculate_F_s = lambda s: 1 + kappa - kappa/(1 + mu*s**2 / kappa)
  s_num   = calculate_s(den_num, grad_den_num)
  F_s_num = calculate_F_s(s_num)

  # LaTeX equation
  equation  = '## $s = \\frac{' + f'|{grad_den_num:.3f}|' + '}{2\cdot3' + '^{1/3}\pi^{2/3}' + f'({den_num:.3f})' + '^{4/3}}=' + f'{s_num:.3f}\ '
  equation += '\ \ \ \ F_{x,\ s=' + f'{s_num:.3f}' + '} = 1 + \kappa - \\frac{\kappa}{(1+\\frac{\mu' + f'({s_num:.3f})^2' + '}{\kappa})}=' + f'{F_s_num:.3f}\ ' + '$'
  equation = Markdown(equation)
  display(equation)
  display(Markdown('<br>'))

  # Matplotlib Diagram
  x_s = np.linspace(0, 10, 300)
  y   = calculate_F_s(x_s)
  plt.plot(x_s, y, label=r'PBE Exch. Factor')
  plt.hlines(F_s_num, 0, s_num, colors='black')
  plt.vlines(s_num, -2, F_s_num, colors='black')
  plt.scatter(s_num, F_s_num, color='red', marker="s", label='grid point')
  plt.title('GGA Enchancement Factor', size=15)
  plt.xlabel('RDG, $s$', size=15)
  plt.ylabel('$F_x(s)$', size=15)
  plt.xlim(0,10)
  plt.ylim(0.95, 2.1)
  plt.legend(loc='upper left', fontsize=15)
  plt.show()

# link sliders to function
GGA_ex_output = widgets.interactive_output(GGAx_pot_eq, {'den_num':GGAx_den_slider,
                                                         'grad_den_num':GGAx_grad_slider})

# generate display
display(widgets.HBox([GGAx_den_slider, GGAx_grad_slider]),
        GGA_ex_output)

# PBE Correlation
Unfortunately, the equations for PBE correlation
are nonintuitive and contain various constants: \
\
$$v_{\text{C}}^{\text{PBE}}(r_s, t) = v_{\text{C}}^{\text{LDA}}(r_s) + H(r_s, t)$$
\
$$H(r_s, t) = \gamma\ln\Big[1+\frac{\beta}{\gamma}t^2\Big(\frac{1+At^2}{1+At^2+A^2t^4}\Big)\Big]$$ \
\
$$ A = \frac{\beta}{\gamma}\Big[\text{exp} \Big\{ \frac{-v_{\text{C}}^{\text{LDA}}}{\gamma \phi^3e^2/a_0} \Big\} - 1 \Big]^{-1} $$
\
$$ t = \frac{|\nabla n(r)|\pi^{1/6}}{4(3)^{1/6}n(r)^{7/6}} $$ \\
What is clear from the above terms is that PBE correlation is defined relative to LDA correlation and is an energy functional of the density. Therefore, given a density and its derivatives the energy from PBE correlation can be determined,
$E_{\text{C}}[n(r), \nabla n(r)] = \int{[v_{\text{C}}^{\text{LDA}}(r_s) + H(r_s, t)]n(r)dr} .$

## Hartree Potential
The Hartree potential, which is a classical potential
that a charge distribution experiences with itself due to Coulomb's law,
has the expression\
\
$$ v_{\text{Ha}}(n(r)) = \int{\frac{n(r')}{|r-r'|}drdr'}.$$ \
This program forgoes the memory-intensive task of directly evaluating this integral
by instead solving Poisson's equation through a Conjuagte Gradient
$Ax = b$ matrix equation solver in scipy given a density, $n(r)$, and finite-difference
representation of $\nabla^2$: \
\
$$ \nabla^2v_{\text{Ha}}(r)=-4\pi n(r) .$$ \
$v_{\text{Ha}}(r)$ can be integrated to yield the (positive) Hartree repulsion energy with a factor of $\frac{1}{2}$ to avoid double counting electrons, $E_{\text{Ha}}[n(r)]=\frac{1}{2}\int{v_{\text{Ha}}(r)n(r)dr}$. As an example, consider below the 1D charge distribution described by
a trigonometric function with some frequency defined from 0 to 2$\pi$.

In [ ]:
#@title **Hartree Potential**

# GUI frequnecy widget
freq_slider = widgets.IntSlider(value=1, min=1, max=10, step=1,
                                description='freq', disabled=False, readout_format='d',
                                continuous_update=False)

def hartree_plotter(freq, solve_poisson):
  '''Display interactive diagram solving for potential from model density in 1D.
  Args:
    freq (int): Coefficient for frequency in trig function.
    solve_poisson (bool): Display (True) potential from trig function.
  '''
  # Data for density and Ax=b solver
  nx     = 300
  x      = np.linspace(0, np.pi, nx)
  y      = np.sin(freq*x)**2
  diag1x = np.ones(nx)/(x[1])
  D1x    = sparse.spdiags(np.array([-diag1x, diag1x]), np.array([0,1]), nx, nx)
  diagx  = np.ones(nx)/(x[1]**2)
  D2x    = sparse.spdiags(np.array([diagx, -2*diagx, diagx]), np.array([-1,0,1]), nx, nx)
  T      = -1/2 * D2x
  test   = sparse.linalg.cg(-2*T, -4.*np.pi*y) # solve for V

  # LaTeX equation
  equation1 = '## $n(x) = \sin^2( 2 \pi ' + f'({freq})' + 'x)$'
  equation2 = '## $\\nabla^2V(x)=-4\pi n(x) \\xrightarrow[\\text{linalg.cg}]{\\text{scipy}} V(x)$'
  display(Markdown(equation1))
  display(Markdown(equation2))

  # Matplotlib Diagram
  plt.gcf()
  plt.clf()
  plt.title('1D Hartree Potential', size=15)
  plt.plot(x, y, label=f'n(x) = $\sin^2(2 \pi ({freq}) x)$')
  if solve_poisson:
    plt.plot(x, test[0], label='V(x)')
  plt.xlabel('x', size=15)
  plt.ylabel('Amplitude', size=15)
  plt.legend(loc='upper right')
  plt.grid()
  plt.show()
  print() # a little extra space

# link sliders to function
hartree_output = widgets.interactive_output(hartree_plotter, {'freq': freq_slider,
                                                              'solve_poisson': widgets.fixed(True)})

# generate the display
display(freq_slider,
        hartree_output)

# Putting Everything Together

With the above definitions we can define an effective **single-particle
Hamiltonian** with kinetic, exchange, correlation,
and Hartree components. This operator will depend on the input density and can be solved with scipy's sparse.linalg.eigsh eigenvalue solver, yielding eigenvalues and eigenvectors corresponding to a new output density, which can then be used to define a new Hamiltonian that can be solved to produce another density, and so on. This cycle will continue until (1) the maximum number of iterations is reached or (2) the energy **convergence threshold** is met, whereby the energy associated with the input and output densities is the same. The input density to these equations at iteration #0 will be the
non-interacting density from the 3D PIB solutions and the final converged density will be optimized according to the Kohn-Sham scheme. Below is a table of parameters for the $\pi$-electron boxes of some polycyclic aromatic hydrocarbons.


<center>

| PAH        | Rings | Dimensions (Bohr) | $\pi$ electrons | Fuse-type    |
|:----------:|:-----:|:-----------------:|:---------------:|:------------:|
| benzene    |   1   |     8 x 8 x 3     |       6         |   linear     |
| naphthalene|   2   |    12 x 8 x 3     |      10         |   linear     |
| anthracene |   3   |    16 x 8 x 3     |      14         |   linear     |
| tetracene  |   4   |    20 x 8 x 3     |      18         |   linear     |
| pentacene  |   5   |    24 x 8 x 3     |      22         |   linear     |
| hexacene   |   6   |    28 x 8 x 3     |      26         |   linear     |
| heptacene  |   7   |    32 x 8 x 3     |      30         |   linear     |
| ---        |  ---  |     ---           |      ---        |     ---      |
| perylene   |   5   |   20 x 12 x 3     |      20         | non-linear   |
| coronene   |   7   |   20 x 16 x 3     |      24         | non-linear   |

<center/>

<p align='left'>
 To ensure stable convergence behavior, an energy threshold of 10$^{-4}$ Ha and 70/30 <b>density mixing</b> is recommended. Density mixing refers to mixing in some fraction of density from the previous iteration to define the new density. A 70/30 linear mixture translates to mixing 70% of the density generated at iteration number $i$ with 30% at iteration $i-1$ to yield a new composite density for iteration $i+1$:
$$ n_{i+1}(r) = 0.7n_i(r) + 0.3n_{i-1}(r) .$$

<p align='left'>
As a <b>word of caution</b>, PBE can become numerically unstable on the grids utilized here, particularly with perfect cubical symmetry. Also, the time required for each iteration scales with the number of electrons and grid size, so if the calculation takes inconveniently long, consider using smaller box parameters such as those in the above table.
</p>

In [ ]:
#@title **DFT Calculator**

### Define GUI widgets
# Box length widgets
length_labels = widgets.Label(value='Box Lengths (Bohr): ')
lx_slider_dft = widgets.IntSlider(value=12,min=3,max=32,step=1,description='lx',disabled=False,readout_format='d',continuous_update=False)
ly_slider_dft = widgets.IntSlider(value=8,min=3,max=32,step=1,description='ly',disabled=False,readout_format='d',continuous_update=False)
lz_slider_dft = widgets.IntSlider(value=3,min=3,max=32,step=1,description='lz',disabled=False,readout_format='d',continuous_update=False)

# Hamiltonian form GUI widgets
functional_dropdown = widgets.Dropdown(options=[('LDA', 'LDA'), ('PBE (GGA)', 'PBE')],
                                       value='LDA',
                                       description='Fuctional:',
                                       disabled=False,
                                       layout = widgets.Layout(width='200px'))
hartree_dropdown    = widgets.Dropdown(options=[('On', True), ('Off', False)],
                                       value=True,
                                       description='Hartree:',
                                       disabled=False,
                                       layout = widgets.Layout(width='175px'))
exchange_dropdown   = widgets.Dropdown(options=[('On', True), ('Off', False)],
                                       value=True,
                                       description='Exchange:',
                                       disabled=False,
                                       layout = widgets.Layout(width='175px'))
correlation_dropdown = widgets.Dropdown(options=[('On', True), ('Off', False)],
                                       value=True,
                                       description='Correlation:',
                                       disabled=False,
                                       layout = widgets.Layout(width='175px'))

# Number of electrons widget
num_elec_label    = widgets.Label(value='Number Electrons: ')
num_elec_dropdown = widgets.Dropdown(options=np.arange(2, 31, 2),
                                    value=10, description=' ',
                                    disabled=False,
                                     layout = widgets.Layout(width='175px'))

converge_parameter_label = widgets.Label(value='Convergence Settings: ')
max_iter_dropdown        = widgets.Dropdown(options=np.arange(5, 105, 5),
                                    value=30, description='max iter',
                                    disabled=False,
                                     layout = widgets.Layout(width='150px'))

density_mix_dropdown     = widgets.Dropdown(options=['OFF', '50/50', '60/40', '70/30', '80/20', '90/10'],
                                       value='70/30',
                                       description='density mix (%): ',
                                       disabled=False,
                                       layout = widgets.Layout(width='200px'),
                                       style = {'description_width': 'initial'})

e_tol_dropdown = widgets.Dropdown(options=['-3', '-4', '-5', '-6'],
                                       value='-4',
                                       description='energy tol (10^ Ha): ',
                                       disabled=False,
                                       layout = widgets.Layout(width='200px'),
                                        style = {'description_width': 'initial'})

# save figure buttons for subsequent analysis cells
filename_text = widgets.Text(description='Filename (.png): ',value='energy_log',style={'description_width': 'initial'})
save_button   = widgets.Button(description='Save Image')

den_filename_text = widgets.Text(description='Filename (.png): ',value='density_slice',style={'description_width': 'initial'})
den_save_button   = widgets.Button(description='Save Image')

orbital_filename_text = widgets.Text(description='Filename (.png): ',value='orbital_energies',style={'description_width': 'initial'})
orbital_save_button   = widgets.Button(description='Save Image')

run_scf_button = widgets.Button(description='Run SCF')
run_scf_button.style.font_weight = 'bold'
reset_button   = widgets.Button(description='Reset')

lib_dropdown = widgets.Dropdown(options=[('ipyvolume', 'ipyvol'), ('plotly', 'plotly')], value='plotly',
                                description='3D Plotting Library: ', disabled=False, layout = widgets.Layout(width='225px'),
                                style = {'description_width': 'initial'})
lib_dropdown.disabled = True

density_analysis_button = widgets.Button(description='Analyze Density')
run_scf_button.style.font_weight = 'bold'

side_select = widgets.SelectionSlider(
options=['lx', 'ly', 'lz'],
    value='lx',
    description='box side: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

scf_ui            = widgets.HBox([run_scf_button, reset_button])
box_length_iso_ui = widgets.HBox([length_labels, lx_slider_dft, ly_slider_dft, lz_slider_dft])
parameter_ui      = widgets.HBox([num_elec_label, num_elec_dropdown])
ham_ui            = widgets.HBox([functional_dropdown, hartree_dropdown,
                                  exchange_dropdown, correlation_dropdown])
convergence_ui    = widgets.HBox([converge_parameter_label, max_iter_dropdown,
                                  density_mix_dropdown, e_tol_dropdown])
filename_text     = widgets.Text(description='Filename (.png): ', value='energy_log',
                                 style={'description_width': 'initial'})
save_button       = widgets.Button(description='Save Image')

def hamiltonian_display(functional, har, ex, cor):
  '''Generates LaTeX equation of effective single-particle Hamiltonian.
  Args:
    functional (str): XC potential (LDA or PBE).
    har (bool): Hartree potential on or off.
    ex (bool): Exchange term on or off.
    cor (bool): Correlation term on or off.
  '''
  ham = '## $$ \hat{h}_i = \hat{T}_{\\text{kin}, i} +'
  if har == True:
    ham += 'v_{\\text{Ha}}(n(r))'
  else:
    ham += '0'
  if ex == True:
    if functional == 'LDA':
      ham += '+ v_{\\text{X}}^{\\text{LDA}}(n(r))'
    elif functional == 'PBE':
      ham += '+ v_{\\text{X}}^{\\text{PBE}}(n(r), \\nabla n(r))'
  else:
    ham += '+ 0'
  if cor == True:
    if functional == 'LDA':
      ham += '+ v_{\\text{C}}^{\\text{LDA}}(n(r))'
    elif functional == 'PBE':
      ham += '+ v_{\\text{C}}^{\\text{PBE}}(n(r), \\nabla n(r))'
  else:
    ham += '+ 0'
  ham += '$$'
  display(Markdown(ham))

output            = widgets.interactive_output(hamiltonian_display, {'functional':functional_dropdown,
                                                 'har':hartree_dropdown,
                                                 'ex':exchange_dropdown,
                                                 'cor':correlation_dropdown})

# various logs to keep tract of SCF loop progression
energy_log     = [0] # zero needed for first loop; removed after end of for loop
ener_diff_log  = []
converged      = None
exch_log       = []
cor_log        = []
har_log        = []
kin_log        = []
eigenvalue_log = []
eigenstate_log = []
density_log    = []
grid_points    = []
grid_lines     = []

def energy_plot(den_log, ener_log, converge_state, show_fig=True, save_fig=False, filename=None):
  '''Generates iteration number vs total energy plot.
  Args:
    den_log (list): List of numpy arrays with electron density.
    ener_log (list): List of total energy values in Ha.
    converge_state (bool): SCF loop ended in a converged (True) or unconverged (False) state.
    show_fig (bool): Display figure to output (True) or not (False).
    save_fig (bool): Save a .png file of diagram.
    filename (str): Filename to save to (w/o extension).
  '''
  fig = plt.figure(figsize=(6, 4))
  ax = fig.add_subplot(1, 1, 1)
  plt.title('Convergence Plot')
  plt.scatter(0, energy_log[0], color='#F97306', label='noninter Energy')
  plt.plot(np.arange(1,len(energy_log[1:]) + 1), energy_log[1:], 'o-', label='DFT Energy')
  plt.legend(loc='upper right')
  plt.text(0.785, 0.800, f'iterations: {len(density_log) - 1}',
          horizontalalignment='center', verticalalignment='center',
          transform=ax.transAxes)
  plt.text(0.785, 0.730,f'            energy (Ha): {energy_log[-1]:.5f}',
          size=10.5, horizontalalignment='center', verticalalignment='center',
          transform= ax.transAxes)
  if converge_state == True:
    plt.text(0.79, 0.660,f'converged',
            size=10.5, horizontalalignment='center', verticalalignment='center',
            transform= ax.transAxes, weight='bold')
  elif converge_state == False:
    plt.text(0.79, 0.660,f'unconverged',
            size=10.5, horizontalalignment='center', verticalalignment='center',
            transform= ax.transAxes, weight='bold')
  plt.ylabel('Energy (Ha)')
  plt.xlabel('iteration #')
  plt.tight_layout()
  if save_fig:
    plt.savefig(f'{filename}.png', dpi = 800)
  if show_fig:
    plt.show()
  else:
    plt.close()

def edge_cleaner(func_3d, nx, ny, nz, num_edges=1):
  '''Sets outermost layer (or two) of grid values of a 3D function to zero.
  This is needed for the density because of the discontinuities in the numeric 2nd derivatives at the box edges.
  Args:
    func_3d (np.array): Flattened 3D grid of points.
    nx, ny, nz (int): Number of grid points in each dimension.
    num_edges (1 or 2): Number of border layers to set to zero.
  Returns:
    A flattened array of grid points.
  '''
  func_3d = func_3d.reshape(nx, ny, nz)
  if num_edges == 1:
    func_3d[0,:,:]  = 0
    func_3d[-1,:,:] = 0
    func_3d[:,0,:]  = 0
    func_3d[:,-1,:] = 0
    func_3d[:,:,0]  = 0
    func_3d[:,:,-1] = 0
  elif num_edges == 2:
    func_3d[0,:,:]  = 0
    func_3d[-1,:,:] = 0
    func_3d[:,0,:]  = 0
    func_3d[:,-1,:] = 0
    func_3d[:,:,0]  = 0
    func_3d[:,:,-1] = 0
    func_3d[1,:,:]  = 0
    func_3d[-2,:,:] = 0
    func_3d[:,1,:]  = 0
    func_3d[:,-2,:] = 0
    func_3d[:,:,1]  = 0
    func_3d[:,:,-2] = 0
  return func_3d.flatten()

def hard_walls(potential, nx, ny, nz):
  '''Sets potential of outermost grid points to 1,000. Used for testing.
  Args:
    potential (np.array): KS (or other) potential as a flattened array.
    nx, ny, nz (int): Grid points in each direction.
  Returns:
    Flattened array of grid points.
  '''
  potential         = potential.reshape(nx, ny, nz)
  potential[0,:,:]  = 1000
  potential[-1,:,:] = 1000
  potential[:,0,:]  = 1000
  potential[:,-1,:] = 1000
  potential[:,:,0]  = 1000
  potential[:,:,-1] = 1000
  return potential.flatten()

def integ_3d(func_3d, dx, dy, dz):
  '''Integrates a 3D function over all defined space.
  Args:
    func_3d (np.array): 3D grid of points.
    dx, dy, dz (float): Differential volume element in each dimension.
  Returns:
    Integrated 3D function as scalar value.
  '''
  return np.sum(func_3d * dx * dy * dz)

def norm_psi_and_den(e_vecs, occ_states, dx, dy, dz):
  '''Normalizes raw eigenvectors from the solver and finds electron density.
  Args:
    e_vecs (np.array): Array of eigenvectors from the eigenvalue solver.
    occ_states (int): Number of occupied KS states (i.e. # electrons / 2).
    dx, dy, dz (float): Differential volume element in each dimension.
  Returns:
    norm_psi (np.array): Array of normalized eigenvectors.
    el_den (np.array): Electron density as array.
  '''
  norm_psi = np.zeros_like(e_vecs)
  el_den   = np.zeros_like(e_vecs[:,0])
  for i in range(e_vecs.shape[1]):
      norm_psi[:,i] = e_vecs[:,i]/np.sqrt(integ_3d(e_vecs[:,i]**2, dx, dy, dz))
  for i in range(occ_states):
      el_den += 2* norm_psi[:,i]**2
  return norm_psi, el_den

def noninter_kin_e(norm_eigenvecs, occ_states, kin_mat, dx, dy, dz, nx, ny, nz):
  '''Finds noninteracting KS kinetic energy.
  Args:
    norm_eigenvecs (np.array): Array of normalized eigenvectors as columns/rows.
    occ_states (int): Number of occupied KS states (i.e. # electrons / 2).
    kin_mat (sparse array): Kinetic energy operator as matrix.
    dx, dy, dz (float): Differential volume element in each dimension.
    nx, ny, nz (int): Number of grid points in each dimension.
  Returns:
    Scalar energy value in Ha.
  '''
  kin_energy_values = []
  for eig in norm_eigenvecs.T[:occ_states]:
    inner_prod   = eig*kin_mat.dot(eig)
    inner_prod   = edge_cleaner(inner_prod, nx, ny, nz, num_edges=1)
    orbital_k_en = integ_3d(inner_prod, dx, dy, dz)
    kin_energy_values.append(orbital_k_en)
  return sum(kin_energy_values)

def hartree(den, kin_oper, dx, dy, dz, nx, ny, nz):
  '''Uses Poisson's equation to find Hartree potential from density.
  Args:
    den (np.array): Electron density.
    kin_oper (np.array): Kinetic energy operator as sparse matrix.
    dx, dy, dz (float): Differential volume element in each dimension.
    nx, ny, nz (int): Number of grid points in each dimension.
  Returns:
    v_ha_flat (np.array): Hartree potential as flattened array.
    v_ha_ener (float): Hartree energy (in Ha); needed to compute total energy.
  '''
  clean_den = np.ma.array(den, mask= abs(den) < 0.000001) # Mask the low-density points
  clean_den = np.ma.filled(clean_den, fill_value=0.0) # Fill with zeros
  den       = clean_den
  den       = edge_cleaner(den, nx, ny, nz, num_edges=1) # Set edges to zero
  v_ha_flat = sparse.linalg.cg(-2*kin_oper,-4.*np.pi*den)[0]
  v_ha_flat = edge_cleaner(v_ha_flat, nx, ny, nz, num_edges=1)
  v_ha_ener = (1/2)*integ_3d(v_ha_flat*den, dx, dy, dz)
  return v_ha_flat, v_ha_ener

def lda_exchange(den, dx, dy, dz):
  '''Finds LDA exchange potential and energy from density.
  Args:
    den (np.array): Electron density.
    dx, dy, dz (float): Differential volume element in each dimension.
  Returns:
    exch_pot_lda (np.array): LDA exchange potential.
    exch_ener_lda (float): LDA exchnage energy (in Ha).
  '''
  exch_pot_lda = -(3/4)*(3/np.pi)**(1/3)*(den)**(1/3)
  clean_den = np.ma.array(den, mask= abs(den) < 0.000001)
  clean_den = np.ma.filled(clean_den, fill_value=0.0)
  den       = clean_den
  exch_ener_lda = -(3/4)*(3/np.pi)**(1/3)*integ_3d(den**(4/3), dx, dy, dz)
  return exch_pot_lda, exch_ener_lda

def lda_correlation(den, dx, dy, dz):
  '''Finds LDA correlation potential and energy from density.
  Args:
    den (np.array): Electron density.
    dx, dy, dz (float): Differential volume element in each dimension.
  Returns:
    corr_pot (np.array): LDA correlation potential.
    corr_en (float): LDA correlation energy (in Ha).
  '''
  # a, b, c fitting constants used in paper: 10.1063/1.4958669
  a, b, c   = (np.log(2)-1)/(2*np.pi**2), 20.4562557, (4*np.pi/3)**(1/3)
  corr_pot  = a*np.log(1 + b*c*den**(1/3) + b*(c**2)*den**(2/3))
  clean_den = np.ma.array(den, mask= abs(den) < 0.000001)
  clean_den = np.ma.filled(clean_den, fill_value=0.0)
  den       = clean_den
  corr_en   = integ_3d(den*corr_pot, dx, dy, dz)
  return corr_pot, corr_en

def RDG(den, der_1st, nx, ny, nz):
  '''Finds dimensionless reduced density gradient (needed for PBE exchange).
  Args:
    den (np.array): Electron density.
    der_1st (np.array): 1st derivative operator as sparse matrix.
    nx, ny, nz (int): Number of grid points in each dimension.
  Returns:
    RDG (np.array): Reduced density gradient as matrix.
  '''
  clean_den = np.ma.array(den, mask = abs(den) < 0.000001)
  den       = clean_den
  RDG       = (2*3**(1/3)*np.pi**(2/3))**(-1) * abs(der_1st.dot(den)) * den**(-4/3)
  RDG       = edge_cleaner(RDG, nx, ny, nz, num_edges=1) #set the edges to zero
  RDG       = np.ma.filled(RDG, fill_value=0.0) # return zeros where density is very small
  return RDG

def pbe_exchange(den, D1st, dx, dy, dz, nx, ny, nz):
  '''Finds PBE exchange potential and energy from density + gradient.
  Args:
    den (np.array): Electron density.
    D1st (np.array): 1st derivative operator as sparse matrix.
    dx, dy, dz (float): Differential volume element in each dimension.
    nx, ny, nz (int): Number of grid points in each dimension.
  Returns:
    exch_pot_pbe (np.array): PBE exchange potential.
    exch_ener_pbe (float): PBE exchange energy (in Ha).
  '''
  # kappa and mu constants used in the paper, 10.1103/PhysRevLett.77.3865
  kappa, mu = 0.804, 0.2195149727645171
  s         = RDG(den, D1st, nx, ny, nz)
  F_xs      = 1 + kappa - kappa * (1 + mu * s**2 / kappa)**(-1) # exch enhancement factor
  exch_pot_pbe  = F_xs * -(3/4)*(3/np.pi)**(1/3)*((den)**(1/3))
  clean_den     = np.ma.array(den, mask= abs(den) < 0.000001)
  clean_den     = np.ma.filled(clean_den, fill_value=0.0)
  den           = clean_den
  exch_ener_pbe = integ_3d(den*exch_pot_pbe, dx, dy, dz)
  return exch_pot_pbe, exch_ener_pbe

def cor_den_grad(den, der_1st, nx, ny, nz):
  '''Finds dimensionless correlation density gradient (used in PBE correlation).
  Args:
    den (np.array): Electron density.
    der_1st (np.array): 1st derivative operator as sparse matrix.
    nx, ny, nz (int): Number of grid points in each dimension.
  Returns:
    t (np.array): Correlation density gradient.
  '''
  d_g       = abs(der_1st.dot(den))
  clean_den = np.ma.array(den, mask = abs(den) < 0.000001)
  den       = clean_den
  t         = (d_g*np.pi**(1/6))/(4*3**(1/6)*den**(7/6))
  t         = edge_cleaner(t, nx, ny, nz, num_edges=1)
  t         = np.ma.filled(t, fill_value=0.0)
  return t

def pbe_correlation(den, der_1st, dx, dy, dz, nx, ny, nz):
  '''Finds PBE correlation potential and energy from density + gradient.
  Args:
    den (np.array): Electron density.
    der_1st (np.array): 1st derivative operator as sparse matrix.
    dx, dy, dz (float): Differential volume element in each dimension.
    nx, ny, nz (int): Number of grid points in each dimension.
  Returns:
    cor_pot_pbe (np.array): PBE correlation potential.
    cor_ener_pbe (float): PBE correlation energy (in Ha).
  '''
  lda_c_pot = lda_correlation(den, dx, dy, dz)[0]
  # beta and gamma constants used in the paper, 10.1103/PhysRevLett.77.3865
  beta, gamma = 0.06672455060314922, 0.031090690869654894
  lda_c_pot = np.ma.array(lda_c_pot, mask = abs(lda_c_pot) < 0.000001)
  A = (beta/gamma)*((np.exp(-lda_c_pot/gamma)-1)**(-1))
  t = cor_den_grad(den, der_1st, nx, ny, nz)
  H = gamma*np.log(1+(beta/gamma)*t**2*((1+A*(t**2))/(1+A*(t**2)+(A**2)*(t**4))))
  cor_pot_pbe = lda_c_pot + H
  cor_ener_pbe = integ_3d(den*cor_pot_pbe, dx, dy, dz)
  return cor_pot_pbe, cor_ener_pbe

def reset(b):
  ''' Resets the calculator with button click. '''
  clear_output()
  # re-enable the GUI
  lx_slider_dft.disabled        = False
  ly_slider_dft.disabled        = False
  lz_slider_dft.disabled        = False
  num_elec_dropdown.disabled    = False
  functional_dropdown.disabled  = False
  hartree_dropdown.disabled     = False
  exchange_dropdown.disabled    = False
  correlation_dropdown.disabled = False
  run_scf_button.disabled       = False
  max_iter_dropdown.disabled    = False
  density_mix_dropdown.disabled = False
  e_tol_dropdown.disabled       = False

  # reset the logs
  global energy_log, ener_diff_log, converged,exch_log, cor_log, har_log, kin_log, eigenvalue_log, eigenstate_log, density_log, grid_points, grid_lines
  energy_log     = [0] # zero needed for first difference; removed after end of for loop
  ener_diff_log  = []
  converged      = False
  exch_log       = []
  cor_log        = []
  har_log        = []
  kin_log        = []
  eigenvalue_log = []
  eigenstate_log = []
  density_log    = []
  grid_points    = []
  grid_lines     = []

  # regenerate the display
  display(Markdown('## <center> Single Particle Hamiltonian <center/> '),
                output,
                Markdown('<br>'),
                ham_ui,
                Markdown('<br>'),
                box_length_iso_ui,
                Markdown('<br>'),
                parameter_ui,
                Markdown('<br>'),
                convergence_ui,
                Markdown('<br>'),
                scf_ui,
                Markdown('---'))

reset_button.on_click(reset) # link button to function

def run_scf(b):
  ''' Run the calculation with widget parameters. '''
  lx_slider_dft.disabled        = True
  ly_slider_dft.disabled        = True
  lz_slider_dft.disabled        = True
  num_elec_dropdown.disabled    = True
  functional_dropdown.disabled  = True
  hartree_dropdown.disabled     = True
  exchange_dropdown.disabled    = True
  correlation_dropdown.disabled = True
  run_scf_button.disabled       = True
  max_iter_dropdown.disabled    = True
  density_mix_dropdown.disabled = True
  e_tol_dropdown.disabled       = True

  occ_states = int(num_elec_dropdown.value/2)
  nextra = 2 # two unoccupied states calculated in addition to occupied (arbitrary number)
  max_iter = int(max_iter_dropdown.value)

  # define the energy tolerance
  e_tol = 10**(int(e_tol_dropdown.value))

  nx, ny, nz = (5 * lx_slider_dft.value), (5 * ly_slider_dft.value), (5 * lz_slider_dft.value)
  grid_points.append([nx, ny, nz])
  xp, yp, zp = np.linspace(0, lx_slider_dft.value, nx), np.linspace(0, ly_slider_dft.value, ny), np.linspace(0, lz_slider_dft.value, nz)
  grid_lines.append([xp, yp, zp])

  # Construction of 1st and 2nd derivative operators through finite-difference method

  diag1x = np.ones(nx)/(xp[1])
  diag1y = np.ones(nx)/(yp[1])
  diag1z = np.ones(nx)/(zp[1])

  # calculation of d/dx, d/dy, and d/dz as sparse matrices:
  D1x = sparse.spdiags(np.array([-diag1x, diag1x]), np.array([0,1]), nx, nx)
  D1y = sparse.spdiags(np.array([-diag1y, diag1y]), np.array([0,1]), ny, ny)
  D1z = sparse.spdiags(np.array([-diag1z, diag1z]), np.array([0,1]), nz, nz)

  # overall 1st derivative for functions on this grid:
  D1st = sparse.kronsum(D1z,sparse.kronsum(D1y,D1x))

  diagx = np.ones(nx)/(xp[1]**2)
  diagy = np.ones(ny)/(yp[1]**2)
  diagz = np.ones(nz)/(zp[1]**2)

  # calculation of d^2/dx^2, d^2/dy^2, and d^2/dz^2 as sparse matrices:
  Dx = sparse.spdiags(np.array([diagx, -2*diagx, diagx]), np.array([-1,0,1]), nx, nx)
  Dy = sparse.spdiags(np.array([diagy, -2*diagy, diagy]), np.array([-1,0,1]), ny, ny)
  Dz = sparse.spdiags(np.array([diagz, -2*diagz, diagz]), np.array([-1,0,1]), nz, nz)

  # construct the sparse matrix
  T = -1/2 * sparse.kronsum(Dz,sparse.kronsum(Dy,Dx))

  columns = ['E_total', 'delta_E', 'E_KE', 'E_x', 'E_c', 'E_ha']
  df = pd.DataFrame(columns =['#i', 'E_tot'.center(7, "-"),
                              'delta_E'.center(7, "-"),
                              'E_KE'.center(7, "-"),
                              'E_x'.center(7, "-"),
                              'E_c'.center(7, "-"),
                              'E_ha'.center(7, "-")])
  display(df) # dataframe with column headings

  # start the SCF loop
  for it_num in range(max_iter + 1):
    if it_num == 0:
      placeholder_density = np.zeros(T.shape[0]) ## placeholder density of zero for the first iteration; removed after loop
      density_log.append(placeholder_density)

    density = density_log[-1]
    density = np.ma.array(density, mask= abs(density) < 0.000001)
    density = np.ma.filled(density, fill_value=0.0)
    density = edge_cleaner(density, nx, ny, nz, num_edges=1)

    ### Hartree on or off
    if hartree_dropdown.value == True:
      if it_num == 0:
        har_pot  = np.zeros(T.shape[0])
        har_ener = 0
      elif it_num != 0:
        har_pot, har_ener = hartree(density, T, xp[1], yp[1], zp[1], nx, ny, nz)
    elif hartree_dropdown.value == False:
      har_pot  = np.zeros(T.shape[0])
      har_ener = 0

    ### exchange lda, pbe, or off
    if exchange_dropdown.value == True:
      if functional_dropdown.value == 'LDA':
        if it_num == 0:
          exch_pot  = np.zeros(T.shape[0])
          exch_ener = 0
        elif it_num != 0:
          exch_pot, exch_ener = lda_exchange(density, xp[1], yp[1], zp[1])
      elif functional_dropdown.value == 'PBE':
        if it_num == 0:
          exch_pot  = np.zeros(T.shape[0])
          exch_ener = 0
        elif it_num != 0:
          exch_pot, exch_ener = pbe_exchange(density, D1st, xp[1], yp[1], zp[1], nx, ny, nz)
    elif exchange_dropdown.value == False:
      exch_pot  = np.zeros(T.shape[0])
      exch_ener = 0

    ### correlation lda, pbe, or off
    if correlation_dropdown.value == True:
      if functional_dropdown.value == 'LDA':
        if it_num == 0:
          cor_pot  = np.zeros(T.shape[0])
          cor_ener = 0
        elif it_num != 0:
          cor_pot, cor_ener = lda_correlation(density, xp[1], yp[1], zp[1])
      elif functional_dropdown.value == 'PBE':
        if it_num == 0:
          cor_pot  = np.zeros(T.shape[0])
          cor_ener = 0
        elif it_num != 0:
          cor_pot, cor_ener = pbe_correlation(density, D1st, xp[1], yp[1], zp[1], nx, ny, nz)
    elif correlation_dropdown.value == False:
      cor_pot  = np.zeros(T.shape[0])
      cor_ener = 0

    v_diagonal = har_pot + exch_pot + cor_pot
    v_clean    = np.ma.array(v_diagonal, mask = abs(density) < 0.000001)
    v_clean    = np.ma.filled(v_clean, fill_value=0.0)

    # write the hamiltonian
    V = sparse.diags(v_clean)

    H = T + V

    # finds the eigenvalues and eigenvectors (k is number of levels)
    eigenenergies, raw_evecs = sparse.linalg.eigsh(H, k=occ_states+nextra, which='SM',mode='cayley')

    normalized_evecs, new_density = norm_psi_and_den(raw_evecs, occ_states, xp[1], yp[1], zp[1])

    new_density = edge_cleaner(new_density, nx, ny, nz, num_edges=1)

    # density mixing section 'OFF', '50/50', '60/40', etc.
    if it_num == 0:
      density_log.append(new_density)
    elif it_num != 0:
      if density_mix_dropdown.value == 'OFF':
        new_new_density = 0.0*density + 1.0*new_density
      elif density_mix_dropdown.value == '50/50':
        new_new_density = 0.5*density + 0.5*new_density
      elif density_mix_dropdown.value == '60/40':
        new_new_density = 0.6*density + 0.4*new_density
      elif density_mix_dropdown.value == '70/30':
        new_new_density = 0.7*density + 0.3*new_density
      elif density_mix_dropdown.value == '80/20':
        new_new_density = 0.8*density + 0.2*new_density
      elif density_mix_dropdown.value == '90/10':
        new_new_density = 0.9*density + 0.1*new_density
      density_log.append(new_new_density)

    # noninteracting kinetic energy
    kin_ener = noninter_kin_e(normalized_evecs, occ_states, T, xp[1], yp[1], yp[1], nx, ny, nz)

    # total KS-DFT energy and delta
    total_ener = kin_ener + har_ener + exch_ener + cor_ener
    delta_ener = total_ener - energy_log[-1]

    # update the logs
    energy_log.append(total_ener)
    ener_diff_log.append(delta_ener)
    kin_log.append(kin_ener)
    exch_log.append(exch_ener)
    cor_log.append(cor_ener)
    har_log.append(har_ener)
    eigenvalue_log.append(eigenenergies)

    dm = pd.DataFrame(columns =[str(it_num).zfill(2), "{:.5f}".format(total_ener).zfill(7),
                                "{:.5f}".format(delta_ener).zfill(7),
                                "{:.5f}".format(kin_ener).zfill(7), "{:.5f}".format(exch_ener).zfill(7),
                                "{:.5f}".format(cor_ener).zfill(7), "{:.5f}".format(har_ener).zfill(7)])
    display(dm) # display total energy and contributions as dataframe

    # Converged exit loop
    if abs(delta_ener) < e_tol:
      converged = True
      latest_energies = eigenvalue_log[-1]
      HL_Gap          = latest_energies[-2] - latest_energies[-3]
      HL_Gap_eV       = round(HL_Gap*27.2114, 3)
      print('   ')
      print('Converged!')
      print(f'delta E = {abs(delta_ener):.6f} < {e_tol:.6f}')
      print(f'HOMO-LUMO Gap is {HL_Gap:.5f} Ha ({HL_Gap_eV:.3f} eV)')
      print(f"Final energy of {energy_log[-1]:.5f} Ha reached in {it_num} iterations")
      break

    # Unconverged exit loop
    if it_num == max_iter:
      converged = False
      print('   ')
      print(f'delta E = {abs(delta_ener):.6f} > {e_tol:.6f}')
      print("Not converged :( ")
      break

  del density_log[0] # remove placeholder values
  del energy_log[0]

  energy_plot(density_log, energy_log, converged)

  def save_energy_plot(b):
    ''' Save a .png file of energy plot. '''
    energy_plot(density_log, energy_log, converged, show_fig=False,
                save_fig=True, filename=filename_text.value)
  save_button.on_click(save_energy_plot)
  display(widgets.HBox([filename_text, save_button]))

run_scf_button.on_click(run_scf) # link scf button + scf function

# generate the display
display(Markdown('## <center> Single Particle Hamiltonian <center/> '),
                output,
                Markdown('<br>'),
                ham_ui,
                Markdown('<br>'),
                box_length_iso_ui,
                Markdown('<br>'),
                parameter_ui,
                Markdown('<br>'),
                convergence_ui,
                Markdown('<br>'),
                scf_ui,
                Markdown('---'))

## Analysis
If a converged solution has been obtained through the above calculator, we can analyze both the energy and density through the below tools. By looking at the 1D planar average of the density along each axis as a function of the iteration number, we can observe where the electron density is moving (if at all) in the course of being optimized. This kind of information about electron redistribution can also be gained through comparing isodensity surfaces of the initial and converged density at the same isovalue.

In [ ]:
#@title **Electron Density Slices**

# making sure the uer has run the SCF loop first
try:
  if len(density_log) != 0:
    color1, color2 = "#D4CC47", "#7C4D8B"

    def hex_to_RGB(hex_str):
      """Convert hex to RGB color."""
      return [int(hex_str[i:i+2], 16) for i in range(1,6,2)]

    def get_color_gradient(c1, c2, n):
      '''Produces a color gradient given two input colors.
      Args:
        c1, c2 ('str'): Colors in hex format.
        n (int): Number of output colors.
      Returns:
        Gradient of n colors.
      '''
      assert n > 1
      c1_rgb = np.array(hex_to_RGB(c1))/255
      c2_rgb = np.array(hex_to_RGB(c2))/255
      mix_pcts = [x/(n-1) for x in range(n)]
      rgb_colors = [((1-mix)*c1_rgb + (mix*c2_rgb)) for mix in mix_pcts]
      return ["#" + "".join([format(int(round(val*255)), "02x") for val in item]) for item in rgb_colors]

    colors = get_color_gradient(color1, color2, len(density_log))

    nx = grid_points[0][0]
    ny = grid_points[0][1]
    nz = grid_points[0][2]

    def density_analysis(index, show_fig=True, save_fig=False, filename=None):
      '''Generates plot of planar average of the electron density.
      Args:
        index (str): Box side to consider, i.e. 'lx', 'ly', or 'lz'.
        show_fig (bool): Whether to display the figure.
        save_fig (bool): Whether to save a .png file.
        filename (str): Filename to save to (w/o) extension.
      '''
      fig = plt.figure()
      ax = fig.add_subplot(1, 1, 1)
      xp = grid_lines[0][0]
      yp = grid_lines[0][1]
      zp = grid_lines[0][2]
      for i in range(len(density_log)):
        density_it = density_log[i].reshape(nx,ny,nz)
        if index == 'lx':
          lin_avg_lx = []
          density_slice = density_it[0,:,:]
          den_slice_sum = np.zeros_like(density_slice)
          for j in range(nx):
            lin_avg_lx.append(density_it[j,:,:].mean())
          plt.plot(xp,lin_avg_lx,label=f'iteration: {i}',color=colors[i])
          plt.xlabel('l$_x$ (Bohr)')
        if index == 'ly':
          lin_avg_ly = []
          density_slice = density_it[:,0,:]
          den_slice_sum = np.zeros_like(density_slice)
          for j in range(ny):
            lin_avg_ly.append(density_it[:,j,:].mean())
          plt.plot(yp,lin_avg_ly,label=f'iteration: {i}',color=colors[i])
          plt.xlabel('l$_y$ (Bohr)')
        if index == 'lz':
          lin_avg_lz = []
          density_slice = density_it[:,:,0]
          den_slice_sum = np.zeros_like(density_slice)
          for j in range(nz):
            lin_avg_lz.append(density_it[:,:,j].mean())
          plt.plot(zp,lin_avg_lz,label=f'iteration: {i}',color=colors[i])
          plt.xlabel('l$_z$ (Bohr)')
      plt.text(1.27, 1.05,f'Final Energy (Ha): {energy_log[-1]:.5f}',
          size=10.5, horizontalalignment='center', verticalalignment='center',
          transform= ax.transAxes)
      plt.title('Planar Average')
      plt.ylabel('Density (e/Bohr$^{3}$)')
      plt.legend(bbox_to_anchor=(1.05, 1.0), ncol=3)

      # control events
      if save_fig:
        plt.savefig(f'{filename}.png', dpi=800)
      if show_fig:
        plt.show()
      else:
        plt.close()

    def save_den_plot(b):
      ''' Button click to save file with widget parameters. '''
      density_analysis(side_select.value, show_fig=False, save_fig=True,
                      filename=den_filename_text.value)

    den_save_button.on_click(save_den_plot) # link button + function

    # link widget to function
    planar_den_output = widgets.interactive_output(density_analysis, {'index':side_select,
                                                                      'show_fig':widgets.fixed(True),
                                                                      'save_fig':widgets.fixed(False),
                                                                      'filename':widgets.fixed(None)})
    # generate display
    display(side_select,
            planar_den_output,
            widgets.HBox([den_filename_text, den_save_button]))

  else:
    display(Markdown('### Please run the above SCF loop first.'))
except NameError:
    display(Markdown('### Please run the above SCF loop first.'))

In [ ]:
#@title **Isodensity Surface Comparison**


# using density_log as a proxy to see if SCF has been run
assert len(density_log) != 0, 'Please run the above SCF loop first.'


display(Markdown('''Here we can compare the initial non-interacting density
with the final converged density. Note that for the
comparison to be meaningful, the same isovalue needs to be used for
both surfaces. Reexecute the cell with CTRL+ENTER if the display breaks.'''))

noninter_den     = density_log[0].reshape(nx,ny,nz)
noninter_den_iso = noninter_den.mean()*2
converged_den    = density_log[-1].reshape(nx,ny,nz)

# Create a subplot with two 3D isosurfaces, vertically oriented
fig = make_subplots(rows=2, cols=1, specs=[[{'type': 'isosurface'}], [{'type': 'isosurface'}]])

fig.update_layout(width=800, height=800)

# Data for the noninteracting density isosurface
xp, yp, zp = grid_lines[0][0], grid_lines[0][1], grid_lines[0][2]
X, Y, Z    = np.meshgrid(xp, yp, zp, indexing='ij')
isosurface_1 = go.Isosurface(
        x=X.flatten(),
        y=Y.flatten(),
        z=Z.flatten(),
        value=noninter_den.flatten(),
        colorscale='BlueRed',
        isomin=-noninter_den_iso,
        isomax=noninter_den_iso,
        surface_count=2,
        showscale=False,
        caps=dict(x_show=False, y_show=False, z_show=False))

# Data for the KS-optimized density isosurface
isosurface_2 = go.Isosurface(
        x=X.flatten(),
        y=Y.flatten(),
        z=Z.flatten(),
        value=converged_den.flatten(),
        colorscale='BlueRed',
        isomin=-noninter_den_iso,
        isomax=noninter_den_iso,
        surface_count=2,
        showscale=False,
        caps=dict(x_show=False, y_show=False, z_show=False))

# Add isosurfaces to the subplot
fig.add_trace(isosurface_1, row=1, col=1)
fig.add_trace(isosurface_2, row=2, col=1)

# Update the layout and add subtitles
fig.update_layout(
    scene=dict(
        xaxis_title='lx',
        yaxis_title='ly',
        zaxis_title='lz',
    ),
    scene2=dict(
        xaxis_title='lx',
        yaxis_title='ly',
        zaxis_title='lz',
    ),
    annotations=[
        dict(
            text=f'Electron density @ <b>iteration 0</b> <br>isovalue: {noninter_den_iso:.5f} e/Bohr<sup>3<sup>',
            x=0.5,
            y=1.1,
            xref='paper',
            yref='paper',
            showarrow=False,
            font=dict(size=18),
        ),
        dict(
            text=f'Electron density @ <b>iteration {len(density_log)-1}</b> <br>isovalue: {noninter_den_iso:.5f} e/Bohr<sup>3<sup>',
            width=800,
            height=400,
            x=0.5,
            y=0.47,
            xref='paper',
            yref='paper',
            showarrow=False,
            font=dict(size=18),
        ),
    ]
)

# Show the subplot
fig.show()


# ipyvolume version of the keep this commented until the library is fixed
#print('\033[1m==============================\033[0m')
#print("\033[1m\033[94mElectron density @ iteration 0\033[0m")
#print('\033[1m==============================\033[0m')
#print(f'isovalue:\033[0m {isoval:.5f} e/Bohr**3')
#ipv.clear()
#fig = ipv.figure(width=500, height=500)
#ipv.plot_isosurface(noninter_den,level=isoval,controls=True,
#                    description='non-interacting density')
#ipv.squarelim()
#ipv.xyzlabel(r'lx','ly','lz')
#ipv.style.box_off()
#ipv.show()
#display(Markdown('---'))
#
## KS-optimized density isosurface
#print('\033[1m==============================\033[0m')
#print(f"\033[1m\033[94mElectron density @ iteration {len(density_log)-1}\033[0m")
#print('\033[1m==============================\033[0m')
#print(f'isovalue:\033[0m {isoval:.5f} e/Bohr**3')
#ipv.clear()
#fig2 = ipv.figure(width=500, height=500)
#ipv.plot_isosurface(converged_den,level=isoval,controls=True,
#                    description='converged density')
#ipv.squarelim()
#ipv.xyzlabel(r'lx','ly','lz')
#ipv.style.box_off()
#ipv.show()


In [ ]:
#@title **Kohn-Sham Orbitals**


# using density_log as a proxy to see if SCF has been run
assert len(density_log) != 0, 'Please run the above SCF loop first.'


display(Markdown('''
The change in energy of the KS orbitals as a function of iteration number
can be realized through this interactive widget.
'''))

# iteration number slider widget
it_num_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(eigenvalue_log)-1,
    step=1,
    description='Iteration #: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# find the max orbital energy for the plot limits
ener_array = np.array(eigenvalue_log)
max_ener   = ener_array.max()
min_ener   = ener_array.min()

def eigenstate_analysis(index, show_fig=True, save_fig=False, filename=None):
  '''Generates an interactive orbital energy diagram from above KS-DFT calculation.
  Args:
    index (int): Iteration number of SCF loop to consider.
    show_fig (bool): Whether to display the figure.
    save_fig (bool): Whether to save a .png file.
    filename (str): Filename to save to (w/o) extension.
  '''
  clear_output(wait=True)
  occ2 = len(eigenvalue_log[0])-2
  y_DFT = eigenvalue_log[index]
  HL_Gap = y_DFT[-2] - y_DFT[-3]
  HL_Gap_eV = round(HL_Gap*27.2114, 3)
  x_DFT = 1.5*np.ones(y_DFT.shape[0])
  fig = plt.figure(figsize=(4, 6))
  ax = fig.add_subplot(1, 1, 1)
  plt.title('Orbital Energies')
  plt.ylabel("Energy (Ha)",labelpad=7)
  plt.scatter(x_DFT[:-2], y_DFT[:-2], marker=0, s=1200, linewidths=4, color='green', label='occupied')
  plt.scatter(x_DFT[-2:], y_DFT[-2:], marker=0, s=1200, linewidths=4, color='#F97306', label='virtual')
  plt.legend(bbox_to_anchor=(1.3, 1))
  plt.rcParams["legend.markerscale"] = 0.35
  plt.xticks([])
  plt.xlim([-0.1, 3.3])
  plt.ylim([min_ener - 0.03, max_ener + 0.03])
  plt.text(1.07, 0.855, f'DFT iteration #{index}',
        horizontalalignment='center', verticalalignment='center',
        transform=ax.transAxes)
  plt.text(1.07, 0.810, f'     Energy: {energy_log[index]:.5f} Ha',
        horizontalalignment='center', verticalalignment='center',
        transform=ax.transAxes)
  plt.text(1.07, 0.765, f'      H-L Gap: {HL_Gap:.4f} Ha',
      horizontalalignment='center', verticalalignment='center',
      transform=ax.transAxes)
  plt.text(1.07, 0.720, f'                       ({HL_Gap_eV:.3f} eV)',
    horizontalalignment='center', verticalalignment='center',
    transform=ax.transAxes)
  annotationsDFT = ['HOMO-' + str(i) for i in range(occ2)]
  annotationsDFT[0] = 'HOMO'
  annotationsDFT.insert(0,'LUMO')
  annotationsDFT.insert(0,'LUMO+1')
  for i, label in enumerate(reversed(annotationsDFT)):
      plt.annotate(label, (x_DFT[i] + 0.005, y_DFT[i]),size=8)
      plt.text(x_DFT[i]-0.9, y_DFT[i], "{:.3f}".format(y_DFT[i]), size=8)
  ax.spines['left'].set_position(('axes', .16))
  ax.spines['top'].set_visible(False)
  ax.spines['right'].set_visible(False)
  ax.spines['bottom'].set_visible(False)

  # control events
  if save_fig:
    plt.tight_layout()
    plt.savefig(f'{filename}.png', dpi=800)
  if show_fig:
    plt.show()
  else:
    plt.close()

def save_orbital_plot(b):
  ''' Saves a .png of orbital energy diagram with widget parameters. '''
  eigenstate_analysis(it_num_slider.value, show_fig=False, save_fig=True,
                      filename=orbital_filename_text.value)

orbital_save_button.on_click(save_orbital_plot) # link button + function

# link widgets to function
eigenstate_output = widgets.interactive_output(eigenstate_analysis, {'index':it_num_slider,
                                                                  'show_fig':widgets.fixed(True),
                                                                  'save_fig':widgets.fixed(False),
                                                                  'filename':widgets.fixed(None)})

# generate display
display(it_num_slider,
        eigenstate_output,
        widgets.HBox([orbital_filename_text, orbital_save_button]))

## **References**

* Hohenberg, P.; Kohn, W. Inhomogeneous Electron Gas. Phys. Rev. **1964**, *136*, B864-B871. DOI: [10.1103/PhysRev.136.B864](https://doi.org/10.1103/PhysRev.136.B864).

*  Kohn, W.; Sham, L. J. Self-Consistent Equations Including Exchange and Correlation Effects. *Physical Review* **1965**, *140*, A1133-A1138. DOI: [10.1103/PhysRev.140.A1133](https://doi.org/10.1103/PhysRev.140.A1133).

*  Baseden, K. A.; Tye, J. W. Introduction to Density Functional Theory: Calculations by Hand on the Helium Atom. *Journal of Chemical Education* **2014**, *91*, 2116–2123. DOI: [10.1021/ed5004788](https://doi.org/10.1021/ed5004788).

*  Chachiyo, T. Communication: Simple and Accurate Uniform Electron Gas Correlation Energy for the Full Range of Densities. *The Journal of Chemical Physics* **2016**, *145*, 021101. DOI: [10.1063/1.4958669](https://doi.org/10.1063/1.4958669).

* Halpern, A. M.; Ge, Y.; Glendening, E. D. Visualizing Solutions of the One-Dimensional Schrödinger Equation Using a Finite Difference Method. *Journal of Chemical Education* **2022**, *99*, 3053–3060. DOI: [10.1021/acs.jchemed.2c00557](https://doi.org/10.1021/acs.jchemed.2c00557).
